In [1]:
import pandas as pd

In [2]:
# Data 불러오기

# data1990 = pd.read_csv("Data/배출원별_온실가스_배출량_1991_2000.csv", encoding='UTF-8',index_col=0)
# data2000 = pd.read_csv("Data/배출원별_온실가스_배출량_2001_2010.csv", encoding='UTF-8',index_col=0)
data2010 = pd.read_csv("Data/배출원별_온실가스_배출량_2011_2021.csv", encoding='UTF-8',index_col=0)

In [3]:
# df1990 = pd.DataFrame(data1990)
# df2000 = pd.DataFrame(data2000)
df2010 = pd.DataFrame(data2010)

In [ ]:
# 데이터를 연도별로 나누기.
def split_dataframe_by_columns(df, num_cols_per_split):
    """
    주어진 데이터프레임을 일정한 열 간격으로 나누어 여러 개의 데이터프레임을 생성하는 함수입니다.

    매개변수:
    df (pd.DataFrame): 원본 데이터프레임
    num_cols_per_split (int): 각 데이터프레임에 포함될 열의 개수

    반환값:
    dict: 키가 'df_연도' 형식의 문자열이고 값이 나뉘어진 데이터프레임인 딕셔너리
    """
    dataframes = {}
    for i in range(0, df.shape[1], num_cols_per_split):
        start_col = i
        end_col = i + num_cols_per_split
        year = 2011 + i // num_cols_per_split
        dataframes[f'df_{year}'] = df.iloc[:, start_col:end_col].copy()
    return dataframes

In [ ]:
# 컬럼명 설정하고 해당 로우 제거.
def set_first_row_as_colnames(df):
    """
    주어진 데이터프레임의 첫 번째 행을 열 이름으로 설정하고 첫 번째 행을 제거하는 함수입니다.

    매개변수:
    df (pd.DataFrame): 원본 데이터프레임

    반환값:
    pd.DataFrame: 첫 번째 행이 열 이름으로 설정된 데이터프레임
    """
    new_df = df.copy()
    new_df.columns = new_df.iloc[0]
    new_df = new_df[1:].reset_index(drop=True)
    return new_df


In [ ]:
def process_dataframe(df):
    """
    데이터프레임에서 모든 값이 NaN인 행의 인덱스를 찾아 대륙 리스트에 저장하고,
    인덱스를 초기화하여 국가를 컬럼으로 설정한 후, 임시 데이터프레임과 대륙 데이터프레임을 반환합니다.

    매개변수:
    df (pd.DataFrame): 원본 데이터프레임

    반환값:
    tuple: (임시 데이터프레임, 대륙 데이터프레임)
    """
    # 모든 값이 NaN인 행의 인덱스를 찾아 대륙(continent)에 넣기
    continent_name = df.index[df.isna().all(axis=1)].tolist()
    
    # 인덱스로 되어있는 국가를 컬럼으로 넣고 인덱스를 숫자로 변경
    df.reset_index(inplace=True)
    
    # 국가가 있는 행만 빼고 나머지 행을 임시로 할당
    temp_df = df.iloc[:, 1:]
    
    # 임시 데이터프레임에서 모든 값이 NaN인 행의 인덱스를 찾아 대륙 인덱스로 설정
    continent_idx = temp_df.index[temp_df.isna().all(axis=1)].tolist()
    
    # 대륙 데이터프레임 생성
    continent_df = pd.DataFrame(continent_idx, index=continent_name, columns=['Index'])
    
    return temp_df, continent_df

In [ ]:


# CSV 파일 읽기
csv_file_path = 'path_to_your_csv_file.csv'  # 파일 경로를 지정하세요.
df = pd.read_csv(csv_file_path)

# 데이터프레임을 연도별로 나누기
dataframes = split_dataframe_by_columns(df, 5)

# 각 데이터프레임의 첫 번째 행을 열 이름으로 설정하고 나머지 작업 수행
for key in dataframes:
    dataframes[key] = set_first_row_as_colnames(dataframes[key])
    temp_df, continent_df = process_dataframe(dataframes[key])
    dataframes[key] = temp_df  # 임시 데이터프레임을 저장 (필요한 경우에 맞게 수정)

    # 연도별 데이터프레임 저장
    temp_df.to_csv(f'{key}.csv', index=False)
    # 대륙 데이터프레임도 저장 (필요한 경우)
    continent_df.to_csv(f'continent_{key}.csv')

# 결과 출력
for key, df in dataframes.items():
    print(f"{key}:")
    print(df.head())  # 데이터프레임의 첫 몇 행을 출력
    print()

---
년도별로 가져온 데이터 나누기

In [4]:
# 첫 5개 열 선택
df_2011 = df2010.iloc[:, 0:5]

# 첫 번째 row의 값을 colnames로 설정
df_2011.columns = df_2011.iloc[0]

# colnames로 해준 row값 제거
df_2011 = df_2011[1:]